## 1. Execute full pipeline

In [1]:
import random
import time

from keras_tuner import RandomSearch

import numpy as np

import pandas as pd

import tensorflow as tf

from config.constants import (
    FORECAST_HORIZON, NB_TRIALS,
    OBSERVATION_WINDOW, SEED, TRAIN_PERC
)

from src.change_point_detector import ChangePointCostFunction, ChangePointMethod, get_change_point_detector
from src.dataset import read_dataset, split_X_y, split_train_test
from src.forecaster import InternalForecaster, TimeSeriesHyperModel
from src.scaler import Scaler
from src.utils import get_error_results

tf.get_logger().setLevel('ERROR')

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


In [2]:
print("Available devices:", tf.config.list_physical_devices())
print("Is GPU available?", tf.config.list_physical_devices('GPU'))


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU available? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
if tf.config.list_physical_devices("GPU"):
    try:
        tf.config.set_visible_devices([], "GPU")  # Hide traditional GPUs if any (like external ones)
        tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[0], True)
        print("✅ TensorFlow is using Apple GPU via MPS")
    except Exception as e:
        print("⚠️ Error setting MPS GPU:", e)
else:
    print("❌ No GPU found, running on CPU")

✅ TensorFlow is using Apple GPU via MPS


In [4]:
timestamp = 'validate_pipeline'
dataset_domain_argv = 'UCI'
dataset_argv = 'AIR_QUALITY'
change_point_method_argv = 'Window'
change_point_cost_function_argv = 'L1'

In [5]:
execution_id = f"{timestamp}_{dataset_domain_argv}_{dataset_argv}_{change_point_method_argv}_{change_point_cost_function_argv}_{SEED}"
change_point_method = ChangePointMethod.from_str(change_point_method_argv)
change_point_cost_function = ChangePointCostFunction.from_str(change_point_cost_function_argv)
change_point_approach = f"{change_point_method.value.title()} {change_point_cost_function.value.title()}"

In [6]:
print(f"[Step 1] Reading dataset {dataset_argv} from {dataset_domain_argv}")
df, variables = read_dataset(dataset_domain_argv, dataset_argv)
print(f"Variables: {variables}")

[Step 1] Reading dataset AIR_QUALITY from UCI
Variables: ['CO(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH']


In [7]:
print("[Step 2] Splitting data into train_val and test")
train_val, test = split_train_test(df)
report = {
    'execution_id': execution_id,
    'timestamp': timestamp,
    'change_point_method': change_point_method.value,
    'change_point_cost_function': change_point_cost_function.value,
    'change_point_approach': change_point_approach,
    'seed': SEED,
    'observation_window': OBSERVATION_WINDOW,
    'train_perc': TRAIN_PERC,
    'nb_trials': NB_TRIALS,
    'dataset_domain': dataset_domain_argv,
    'dataset': dataset_argv,
    'variables': variables,
    'dataset_shape': df.shape,
    'train_val_shape': train_val.shape,
    'test_shape': test.shape,
}

[Step 2] Splitting data into train_val and test


In [8]:
print(f"[Step 3] Detecting cut point ({change_point_approach})")
start_time = time.time()
change_point_detector = get_change_point_detector(change_point_method, change_point_cost_function)
change_point, change_point_perc = change_point_detector.find_change_point(train_val, variables)
end_time = time.time()
detect_change_point_duration = end_time - start_time
print(f"Change point: {change_point}, Change point percentage: {change_point_perc}")
report.update({
    'detect_change_point_duration': detect_change_point_duration,
    'change_point': str(change_point),
    'change_point_perc': change_point_perc
})

[Step 3] Detecting cut point (Window L1)
Change point: 5385, Change point percentage: 71.9438877755511


In [9]:
print("[Step 4] Reducing train_val based on change point")
start_time = time.time()
reduced_train_val = change_point_detector.apply_change_point(train_val, change_point)
end_time = time.time()
apply_change_point_duration = end_time - start_time
report.update({
    'apply_change_point_duration': apply_change_point_duration,
    'reduced_train_val.shape': reduced_train_val.shape,
})

[Step 4] Reducing train_val based on change point


In [10]:
print("[Step 5] Splitting train_val into train and val")
reduced_train, reduced_val = split_train_test(reduced_train_val)
report.update({
    'reduced_train.shape': reduced_train.shape,
    'reduced_val.shape': reduced_val.shape,
})

[Step 5] Splitting train_val into train and val


In [11]:
print("[Step 6] Fitting scaler on train and applying on train and val")
start_time = time.time()
scaler = Scaler(variables)
scaled_reduced_train = scaler.fit_scale(reduced_train)
scaled_reduced_val = scaler.scale(reduced_val)
end_time = time.time()
fit_apply_scaler_train_val_duration = end_time - start_time
report.update({
    'fit_apply_scaler_train_val_duration': fit_apply_scaler_train_val_duration,
})

[Step 6] Fitting scaler on train and applying on train and val


In [12]:
print("[Step 7] Splitting train and val into X and y")
X_reduced_scaled_train, y_reduced_scaled_train = split_X_y(scaled_reduced_train)
X_reduced_scaled_val, y_reduced_scaled_val = split_X_y(scaled_reduced_val)
report.update({
    'X_reduced_scaled_train.shape': X_reduced_scaled_train.shape,
    'y_reduced_scaled_train.shape': y_reduced_scaled_train.shape,
    'X_reduced_scaled_val.shape': X_reduced_scaled_val.shape,
    'y_reduced_scaled_val.shape': y_reduced_scaled_val.shape,
})

[Step 7] Splitting train and val into X and y


In [13]:
y_reduced_scaled_train[0]

array([[ 0.2794384 ,  0.94363348, -0.22221197, -0.31901199,  2.57306525,
        -0.60601449],
       [ 0.93656296,  1.80147318,  0.20238044,  0.04013744,  2.3587538 ,
        -0.29619734],
       [ 1.81272904,  2.47234781,  0.6914199 ,  0.08239032,  2.20289093,
        -0.06383447],
       [ 1.26512524,  1.66949784,  0.35022958, -0.19225337,  2.12495949,
        -0.10901614],
       [ 0.88180258,  1.38355127,  0.17205241, -0.25563268,  1.91064804,
         0.1168922 ],
       [ 0.88180258,  1.30656566,  0.25924549, -0.53027636,  1.67685373,
         0.43316387],
       [ 0.8270422 ,  1.08660677,  0.17963441, -0.53027636,  1.52099086,
         0.56870888]])

In [14]:
y_reduced_scaled_train[1]

array([[ 0.93656296,  1.80147318,  0.20238044,  0.04013744,  2.3587538 ,
        -0.29619734],
       [ 1.81272904,  2.47234781,  0.6914199 ,  0.08239032,  2.20289093,
        -0.06383447],
       [ 1.26512524,  1.66949784,  0.35022958, -0.19225337,  2.12495949,
        -0.10901614],
       [ 0.88180258,  1.38355127,  0.17205241, -0.25563268,  1.91064804,
         0.1168922 ],
       [ 0.88180258,  1.30656566,  0.25924549, -0.53027636,  1.67685373,
         0.43316387],
       [ 0.8270422 ,  1.08660677,  0.17963441, -0.53027636,  1.52099086,
         0.56870888],
       [-0.26816541,  0.09679173, -0.64680438, -0.88942579,  1.46254228,
         0.52352721]])

In [15]:
y_reduced_scaled_train[2]

array([[ 1.81272904,  2.47234781,  0.6914199 ,  0.08239032,  2.20289093,
        -0.06383447],
       [ 1.26512524,  1.66949784,  0.35022958, -0.19225337,  2.12495949,
        -0.10901614],
       [ 0.88180258,  1.38355127,  0.17205241, -0.25563268,  1.91064804,
         0.1168922 ],
       [ 0.88180258,  1.30656566,  0.25924549, -0.53027636,  1.67685373,
         0.43316387],
       [ 0.8270422 ,  1.08660677,  0.17963441, -0.53027636,  1.52099086,
         0.56870888],
       [-0.26816541,  0.09679173, -0.64680438, -0.88942579,  1.46254228,
         0.52352721],
       [-0.76100883, -0.48609934, -1.01453173, -1.1218166 ,  1.44305942,
         0.52998174]])

In [16]:
print("[Step 8] Running HPO and NAS")
n_variables = len(variables)
forecaster_hypermodel = TimeSeriesHyperModel(
    n_variables=n_variables
)
forecaster_tuner = RandomSearch(
    forecaster_hypermodel,
    objective='val_loss',
    max_trials=3,
    executions_per_trial=1,
    directory=f"outputs/tuner/delete_me",
    project_name="delete_me",
    seed=SEED,
    overwrite=True,
    distribution_strategy=tf.distribute.MirroredStrategy()
)
start_time = time.time()
forecaster_tuner.search(
    X_reduced_scaled_train,
    y_reduced_scaled_train,
    validation_data=(X_reduced_scaled_val, y_reduced_scaled_val),
    shuffle=False,
)
end_time = time.time()
tuner_duration = end_time - start_time
report.update({
    'tuner_duration': tuner_duration
})

Trial 3 Complete [00h 00m 05s]
val_loss: 0.40178698493588355

Best val_loss So Far: 0.40178698493588355
Total elapsed time: 00h 00m 28s


In [17]:
print("[Step 9] Retrieving best model")
best_trial = forecaster_tuner.oracle.get_best_trials(num_trials=1)[0]
best_forecaster_model = forecaster_tuner.get_best_models(num_models=1)[0]
print(f"Trial ID: {best_trial.trial_id}")
print(f"Hyperparameters: {best_trial.hyperparameters.values}")
print(f"Score: {best_trial.score}")
print("-" * 40)
best_forecaster_model.summary()
best_forecaster_model = InternalForecaster(
    best_forecaster_model,
    len(variables),
    best_trial.hyperparameters.values['batch_size'],
    best_trial.hyperparameters.values['epochs'],
)
report.update({
    'best_trial_id': best_trial.trial_id,
    'best_trial_hyperparameters': best_trial.hyperparameters.values,
    'best_trial_score': best_trial.score,
    'best_forecaster_model': best_forecaster_model.summary(),
})

[Step 9] Retrieving best model
Trial ID: 2
Hyperparameters: {'num_layers': 1, 'units_0': 96, 'learning_rate': 0.001, 'units_1': 64, 'units_2': 128, 'batch_size': 64, 'epochs': 125, 'units_3': 128, 'units_4': 64}
Score: 0.40178698493588355
----------------------------------------


/Users/zamith/Doutorado/TSForecastingSubsets/env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 96)             │        39,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 96)             │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 42)             │         4,074 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 6)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,010 (171.91 KB)

 Trainable params: 43,818 (171.16 KB)

 Non-trainable params: 192 (768.00 B)

In [18]:
print("[Step 10] Fitting scaler on train_val and applying on train_val and test")
start_time = time.time()
scaler = Scaler(variables)
scaled_reduced_train_val = scaler.fit_scale(reduced_train_val)
scaled_test = scaler.scale(test)
end_time = time.time()
fit_apply_scaler_train_val_test_duration = end_time - start_time
report.update({
    'fit_apply_scaler_train_val_test_duration': fit_apply_scaler_train_val_test_duration,
})

[Step 10] Fitting scaler on train_val and applying on train_val and test


In [19]:
print("[Step 11] Splitting train_val and test into X and y")
X_reduced_scaled_train_val, y_reduced_scaled_train_val = split_X_y(scaled_reduced_train_val)
X_scaled_test, y_scaled_test = split_X_y(scaled_test)
report.update({
    'X_reduced_scaled_train_val.shape': X_reduced_scaled_train_val.shape,
    'y_reduced_scaled_train_val.shape': y_reduced_scaled_train_val.shape,
    'X_scaled_test.shape': X_scaled_test.shape,
    'y_scaled_test.shape': y_scaled_test.shape,
})

[Step 11] Splitting train_val and test into X and y


In [20]:
print("[Step 12] Retraining best model")
start_time = time.time()
best_forecaster_model.fit(
    X_reduced_scaled_train_val,
    y_reduced_scaled_train_val,
    shuffle=False
)
end_time = time.time()
retrain_duration = end_time - start_time
report.update({
    'retrain_duration': retrain_duration,
})

[Step 12] Retraining best model
Epoch 1/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.7986 
Epoch 2/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7762
Epoch 3/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7292
Epoch 4/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6785
Epoch 5/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6503
Epoch 6/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6128
Epoch 7/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6127
Epoch 8/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6357
Epoch 9/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6109
Epoch 10/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5855
Epoch 11/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6301
Epoch 12/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6308
Epoch 13/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6174
Epoch 14/125
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6704
Epoch 15/125
32/32 ━━━━━━━━━━━━━━━━━━━

In [21]:
print("[Step 13] Forecasting for test")
start_time = time.time()
y_scaled_pred = best_forecaster_model.forecast(X_scaled_test)
y_scaled_test_flat = y_scaled_test.reshape(-1, n_variables)
y_scaled_pred_flat = y_scaled_pred.reshape(-1, n_variables)
end_time = time.time()
forecasting_test_duration = end_time - start_time
report.update({
    'forecasting_test_duration': forecasting_test_duration,
})

[Step 13] Forecasting for test
42/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  

2025-04-06 17:57:03.983138: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2025-04-06 17:57:04.162422: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [22]:
y_scaled_test_flat

array([[-1.23055822, -1.25086781, -1.16213415, -0.87719569, -2.06214374,
        -0.13780333],
       [-1.11462147, -1.18137868, -1.05524888, -0.6343508 , -1.94346707,
        -0.31248902],
       [-0.94071634, -0.99607432, -1.01566174, -0.52396676, -1.86434929,
        -0.3528011 ],
       ...,
       [-0.12915906,  0.06942571, -0.37435011,  1.24217789,  2.74426146,
        -2.84543152],
       [-0.3030642 , -0.26643843, -0.60395551,  0.82271854,  3.0211737 ,
        -3.16792817],
       [-0.24509582,  0.0115181 , -0.4851941 ,  1.08764024,  3.06073259,
        -3.1948029 ]])

In [23]:
y_scaled_pred_flat

array([[-1.1374776 , -1.012333  , -0.76727474, -1.1795882 , -1.4156504 ,
        -0.31508484],
       [-1.082217  , -0.648935  , -0.6785568 , -1.0851161 , -1.1200556 ,
        -0.34968984],
       [-0.77705866, -0.49276745, -0.36904395, -0.526343  , -1.188053  ,
        -0.64627653],
       ...,
       [ 0.1366496 ,  0.22431068,  0.32313225,  0.7002863 ,  0.69994354,
        -1.0550631 ],
       [ 0.23887031,  0.45166597,  0.2055144 ,  0.53611845,  0.58992374,
        -1.1682652 ],
       [ 0.26597652,  0.57714087,  0.06440331,  0.6692123 ,  0.64261   ,
        -1.04964   ]], dtype=float32)

In [24]:
print("[Step 14] Descaling data")
start_time = time.time()
y_test = scaler.descale(pd.DataFrame(y_scaled_test_flat, columns=variables))
y_pred = scaler.descale(pd.DataFrame(y_scaled_pred_flat, columns=variables))
end_time = time.time()
descaling_duration = end_time - start_time
report.update({
    'descaling_duration': descaling_duration,
})

[Step 14] Descaling data


In [25]:
y_test

,CO(GT),C6H6(GT),NOx(GT),NO2(GT),T,RH
0,0.5,1.0,94.0,79.0,2.6,58.6
1,0.7,1.6,121.0,90.0,3.2,56.0
2,1.0,3.2,131.0,95.0,3.6,55.4
3,3.0,15.9,552.0,169.0,1.8,63.3
4,4.1,18.0,614.0,199.0,2.4,60.2
...,...,...,...,...,...,...
12959,3.1,13.5,472.0,190.0,21.9,29.3
12960,2.4,11.4,353.0,179.0,24.3,23.7
12961,2.4,12.4,293.0,175.0,26.9,18.3
12962,2.1,9.5,235.0,156.0,28.3,13.5


In [26]:
y_pred

,CO(GT),C6H6(GT),NOx(GT),NO2(GT),T,RH
0,0.660571,3.059615,193.744370,65.302704,5.868511,55.961365
1,0.755900,6.197355,216.155167,69.581947,7.362966,55.446308
2,1.282322,7.545775,294.340363,94.892365,7.019187,51.031948
3,1.908177,7.248773,363.813324,96.972267,9.014489,50.858578
4,2.315074,11.624516,423.422363,128.340439,9.797011,47.230328
...,...,...,...,...,...,...
12959,3.638117,15.591415,569.877319,173.803497,14.393326,47.476746
12960,3.548509,14.566921,532.971924,158.348434,16.147224,45.591976
12961,2.858541,13.737346,469.189148,150.454254,16.564432,44.947620
12962,3.034879,15.700432,439.477997,143.018051,16.008200,43.262733


In [27]:
print("[Step 15] Calculating evaluation metrics")
total_duration = sum(value for key, value in report.items() if key.endswith('_duration'))
error_results = get_error_results(y_test, y_pred, variables)
print(f"Obtained error results: {error_results}")
report.update({
    'total_duration': total_duration,
    'error_results': error_results,
})

[Step 15] Calculating evaluation metrics
Obtained error results: {'Avg_MAPE': 3524438711177.57, 'Avg_MAE': 35.478859416199846, 'Avg_MSE': 6905.7638032438335, 'Avg_RMSE': 44.756211716312684, 'Avg_R2': 0.23097887983504264, 'Avg_WAPE': 0.4144975764736879, 'CO(GT)_MAPE': 1.2771239351480383, 'CO(GT)_MAE': 1.0536267763663496, 'CO(GT)_MSE': 1.7746048470120002, 'CO(GT)_RMSE': 1.3321429529190927, 'CO(GT)_R2': 0.07558978906811242, 'CO(GT)_WAPE': 0.5549944244980833, 'C6H6(GT)_MAPE': 1.658938337720788, 'C6H6(GT)_MAE': 5.091344981404542, 'C6H6(GT)_MSE': 40.4341111614347, 'C6H6(GT)_RMSE': 6.358782207422637, 'C6H6(GT)_R2': 0.03357245801894693, 'C6H6(GT)_WAPE': 0.6363362123042099, 'NOx(GT)_MAPE': 0.9197530595059527, 'NOx(GT)_MAE': 158.12184955503082, 'NOx(GT)_MSE': 39027.52857904098, 'NOx(GT)_RMSE': 197.55386247563214, 'NOx(GT)_R2': 0.10895715821546503, 'NOx(GT)_WAPE': 0.5249694176683876, 'NO2(GT)_MAPE': 0.28087369777296467, 'NO2(GT)_MAE': 35.61151894313691, 'NO2(GT)_MSE': 2201.857948240159, 'NO2(GT)_

In [28]:
report

{'execution_id': 'validate_pipeline_UCI_AIR_QUALITY_Window_L1_214',
 'timestamp': 'validate_pipeline',
 'change_point_method': 'Window',
 'change_point_cost_function': 'L1',
 'change_point_approach': 'Window L1',
 'seed': 214,
 'observation_window': 14,
 'train_perc': 0.8,
 'nb_trials': 30,
 'dataset_domain': 'UCI',
 'dataset': 'AIR_QUALITY',
 'variables': ['CO(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH'],
 'dataset_shape': (9357, 7),
 'train_val_shape': (7485, 7),
 'test_shape': (1872, 7),
 'detect_change_point_duration': 0.07590699195861816,
 'change_point': '5385',
 'change_point_perc': 71.9438877755511,
 'apply_change_point_duration': 8.320808410644531e-05,
 'reduced_train_val.shape': (2100, 7),
 'reduced_train.shape': (1680, 7),
 'reduced_val.shape': (420, 7),
 'fit_apply_scaler_train_val_duration': 0.00370025634765625,
 'X_reduced_scaled_train.shape': (1660, 14, 6),
 'y_reduced_scaled_train.shape': (1660, 7, 6),
 'X_reduced_scaled_val.shape': (400, 14, 6),
 'y_reduced_scale

## 2. What would be the error if we predicted the average values for all variables (Dummy Forecaster)?

In [29]:
X_train_val, y_train_val = split_X_y(train_val)
X_test, y_test = split_X_y(test)

In [30]:
train_val_targets_flat = pd.DataFrame(y_train_val.reshape(-1, len(variables)), columns=variables)
avg_values = train_val_targets_flat.mean(axis=0).to_numpy()

In [31]:
n_test = y_test.shape[0]
dummy_pred = np.tile(avg_values, (n_test, FORECAST_HORIZON, 1))

dummy_pred_flat = dummy_pred.reshape(-1, n_variables)
y_test_flat = pd.DataFrame(y_test.reshape(-1, n_variables), columns=variables)

In [32]:
dummy_error_results = get_error_results(y_test_flat, dummy_pred_flat, variables)
print(f"Error metrics for Dummy Forecaster (predicting average values): \n{dummy_error_results}")

Error metrics for Dummy Forecaster (predicting average values): 
{'Avg_MAPE': 8234635315137.397, 'Avg_MAE': 39.6761911395693, 'Avg_MSE': 9050.378952062862, 'Avg_RMSE': 54.45008211461132, 'Avg_R2': -0.6663866828639414, 'Avg_WAPE': 0.4635347737122664, 'CO(GT)_MAPE': 1.4907392486029176, 'CO(GT)_MAE': 1.1496764140739417, 'CO(GT)_MSE': 2.0059470835605415, 'CO(GT)_RMSE': 1.4163146131988265, 'CO(GT)_R2': -0.044918799672288845, 'CO(GT)_WAPE': 0.6055882539246805, 'C6H6(GT)_MAPE': 2.4158267445655204, 'C6H6(GT)_MAE': 5.998942145850846, 'C6H6(GT)_MSE': 49.37824329877635, 'C6H6(GT)_RMSE': 7.026965440271949, 'C6H6(GT)_R2': -0.18020386569282976, 'C6H6(GT)_WAPE': 0.7497712562918357, 'NOx(GT)_MAPE': 0.69058204879659, 'NOx(GT)_MAE': 152.8286100552228, 'NOx(GT)_MSE': 49209.25736669915, 'NOx(GT)_RMSE': 221.8315968627985, 'NOx(GT)_R2': -0.12350328403007538, 'NOx(GT)_WAPE': 0.5073956992631623, 'NO2(GT)_MAPE': 0.38503788600654404, 'NO2(GT)_MAE': 53.84588657162112, 'NO2(GT)_MSE': 4627.602712434949, 'NO2(GT)_R

In [33]:
df_comparison = pd.DataFrame({
    "Trained Model": pd.Series(error_results),
    "Dummy Forecaster": pd.Series(dummy_error_results)
})

df_comparison = df_comparison.round(5)
df_comparison

,Trained Model,Dummy Forecaster
Avg_MAPE,3.524439e+12,8.234635e+12
Avg_MAE,3.547886e+01,3.967619e+01
Avg_MSE,6.905764e+03,9.050379e+03
Avg_RMSE,4.475621e+01,5.445008e+01
Avg_R2,2.309800e-01,-6.663900e-01
Avg_WAPE,4.145000e-01,4.635300e-01
CO(GT)_MAPE,1.277120e+00,1.490740e+00
CO(GT)_MAE,1.053630e+00,1.149680e+00
CO(GT)_MSE,1.774600e+00,2.005950e+00
CO(GT)_RMSE,1.332140e+00,1.416310e+00
